## Name & ID

In [71]:
student_name = "Mehdi Feghhi"
student_id = 401722136

## Import libraries 

In [72]:
import numpy as np
import re
import string
import random

# Implementation section

## part a

AT First, I replace punctuation with space punctuation space .
Then I split word by space AND half space to token the sentences.
And I return tokens

In [73]:
def tokenize(text):

    for punc in string.punctuation:
        text = text.replace(punc,' '+punc+' ')

    text_without_space = re.split(" |\u200c",text)

    return text_without_space

## part b

At first, I add one <END> at end of sentence tokens then I add (n-1) <start> add first of tokens n is number of grams.
Then from first taken after <start> I found all n-1 tokens before it and put it on list then I make it to tuple like return of sample.
this list include n-1 tuple tokens before this token and token.

In [74]:
def ngrams(n, tokens):
    tokens.append('<END>')
    for i in range(n-1):
        tokens = ['<START>']+ tokens
    # print(tokens)
    list_ngram = []
    for i in range(n-1,len(tokens)):
        # print(i)
        item_before = []
        for j in range(n-1):
            # print(j)
            item_before = [tokens[i-j-1]]+ item_before
        list_ngram.append((tuple(item_before),tokens[i]))

    return list_ngram

## NgramModel class

In [75]:
class NgramModel(object):

    def __init__(self, n):
        self.n = n
        self.res_str =[]

    '''
        res_str include ngrams of each sentence as input we use ngrams function in this section
    '''
    def update(self, sentence):
        self.res_str += ngrams(self.n,tokenize(sentence))

    '''
        for calculate prob of each token with its context .
        at first must calculate number of context in the all res_str as denominator.
        then calculate number of tokens come after this context as numerator.
        at end we return division of numerator and denominator for probability .
    '''

    def prob(self, context, token):

        denominator = 0
        numerator = 0
        # print(self.res_str)
        for item in self.res_str:
            if item[0] == context:
                denominator += 1
                if item[1] == token:
                    numerator += 1

        return numerator/denominator

    '''
        Find all token that come with context
        I sort it by alphabet .
        then I find random.number
        then I find word that sum of the prob of word before it with this token upper than r .
        then I find prob of each item .

    '''

    def random_token(self, context):

        r = random.random()
        map_to_probs = {}
        token_of_interest = [item[1] for item in self.res_str if item[0]==context]
        for token in token_of_interest:
            map_to_probs[token] = self.prob(context, token)

        summ = 0
        for token in sorted(map_to_probs):
            summ += map_to_probs[token]
            if summ > r:
                return token
        return "<END>"

    """
    I generate a text with number token function give with random_token.
    for better result I make context with use each word I generate with random_token.
    """
    def random_text(self, token_count):

        list_item = []
        "add n-1 <start> for context for first context"
        new_context = ["<START>" for i in range(self.n-1)]

        # new_context = tuple(new_context)
        for count in range(token_count):

            find_item = self.random_token(tuple(new_context))
            list_item.append(find_item)
            if self.n > 1 :
                if find_item == '<END>':
                    "if find <end> as item I want to make new sentence because of that change the new context like first one ."
                    new_context = ["<START>" for i in range(self.n-1)]
                else:
                    "update new context with new word we find "
                    new_context.pop(0)
                    new_context.append(find_item)

        return " ".join(list_item)

    """
    calculate perplexity like formulation . at first I find ngram list then I calculate prob of each token and then mul it with res mul of befor token in sentence.
    """
    def perplexity(self, sentence):
        ngram_list = ngrams(self.n,tokenize(sentence))
        res_mul = 1
        for item in ngram_list:
            res_mul = res_mul * self.prob(item[0],item[1])

        return (1/res_mul)**(1/len(ngram_list))


## part f

With this function I read all sentence of text file and update of ngram model that make with n.

In [124]:
def create_ngram_model(n, path):
    model = NgramModel(n)
    file1 = open(path, 'r')
    lines = file1.readlines()

    # x = 0
    # list_add = []
    for line in lines:
         model.update(line.strip())
    return model

# Check section

## part a

In [77]:
tokenize("منم کاوه دادخواه یکی بی‌زیان مرد آهنگرم")

['منم', 'کاوه', 'دادخواه', 'یکی', 'بی', 'زیان', 'مرد', 'آهنگرم']

In [78]:
tokenize("I'm Kaveh, and a blacksmith, sire")

['I', "'", 'm', 'Kaveh', ',', '', 'and', 'a', 'blacksmith', ',', '', 'sire']

## part b

In [79]:
ngrams(1, ["کاوه", "اهنگر", "دادخواه"])

[((), 'کاوه'), ((), 'اهنگر'), ((), 'دادخواه'), ((), '<END>')]

In [80]:
ngrams(2, ["کاوه", "اهنگر", "دادخواه"])

[(('<START>',), 'کاوه'),
 (('کاوه',), 'اهنگر'),
 (('اهنگر',), 'دادخواه'),
 (('دادخواه',), '<END>')]

In [81]:
ngrams(3, ["کاوه", "اهنگر", "دادخواه"])

[(('<START>', '<START>'), 'کاوه'),
 (('<START>', 'کاوه'), 'اهنگر'),
 (('کاوه', 'اهنگر'), 'دادخواه'),
 (('اهنگر', 'دادخواه'), '<END>')]

## part c

In [82]:
m = NgramModel(1)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
print(m.prob((), "کاوه"))
print(m.prob((), "ضحاک"))
print(m.prob((), "<END>"))

0.3
0.1
0.2


In [83]:
m = NgramModel(2)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
print(m.prob(("<START>",), "کاوه"))
print(m.prob(("آهنگر",), "ضحاک"))
print(m.prob(("کاوه",), "ماردوش"))

1.0
0.3333333333333333
0.0


## part d

In [84]:
m = NgramModel(1)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
random.seed(1)
[m.random_token(()) for i in range(8)]

['<END>', 'کاوه', 'کاوه', 'آهنگر', 'آهنگر', 'آهنگر', 'ضحاک', 'کاوه']

In [85]:
m = NgramModel(2)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
random.seed(2)
print([m.random_token(("<START>",)) for i in range(6)])
print([m.random_token(("آهنگر",)) for i in range(6)])

['کاوه', 'کاوه', 'کاوه', 'کاوه', 'کاوه', 'کاوه']
['کاوه', '<END>', 'ضحاک', 'ضحاک', 'ضحاک', '<END>']


## part e

In [86]:
m = NgramModel(1)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
random.seed(1)
m.random_text(16)

'<END> کاوه کاوه آهنگر آهنگر آهنگر ضحاک کاوه <END> <END> کاوه آهنگر کاوه <END> آهنگر کاوه'

In [87]:
m = NgramModel(2)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
random.seed(2)
m.random_text(16)

'کاوه آهنگر <END> کاوه آهنگر کاوه آهنگر <END> کاوه آهنگر ضحاک ستمگر <END> کاوه آهنگر کاوه'

## part f

In [123]:
# No random seeds, so your results may vary
m = create_ngram_model(1, "Shahnameh.txt") 
m.random_text(16)

'<END> to blow nobleman and ” seal he , aside  him this . drew city'

In [90]:
# No random seeds, so your results may vary
m = create_ngram_model(1, "Zahhak.txt") 
m.random_text(16)

'یکی زیان <END> پیکر نهادند فرخ چو مردم نبایدت آمدش گوا گذشتن <END> ماند شاهی برآمد'

In [91]:
# No random seeds, so your results may vary
m = create_ngram_model(1, "Akhavan.txt") 
m.random_text(16)

'خاکستری بی کشد ته و است وز پرده برف درون کرد شکاف بی <END> آهسته سیاه'

In [125]:
# No random seeds, so your results may vary
m = create_ngram_model(2, "Shahnameh.txt") 
m.random_text(16)

'Then on us now conquered my father had looked on their chests ,  branches of'

In [93]:
# No random seeds, so your results may vary
m = create_ngram_model(2, "Zahhak.txt") 
m.random_text(16)

'که بر و زد دست بردند گرز پیکر بدیشان نمود <END>  <END> که ضحاک بیرون'

In [94]:
# No random seeds, so your results may vary
m = create_ngram_model(2, "Akhavan.txt") 
m.random_text(16)

'چه عمر راحتی دنیای خوبی <END> نه حتی جنگلی کوچک ، خون گرگان  . '

In [117]:
# No random seeds, so your results may vary
m = create_ngram_model(3, "Shahnameh.txt") 
m.random_text(16)

' <END>  <END> this you will attain to your love he <END> brought a large'

In [96]:
# No random seeds, so your results may vary
m = create_ngram_model(3, "Zahhak.txt") 
m.random_text(16)

'ز ایوان برون شد ز درگاه شاه <END> ز ایوان برون شد خروشان به کوی <END>'

In [97]:
# No random seeds, so your results may vary
m = create_ngram_model(3, "Akhavan.txt") 
m.random_text(16)

' <END> بلای نیستی ، سرمای پر سوز <END>  <END> ولی شلاق  ! '

In [116]:
# No random seeds, so your results may vary
m = create_ngram_model(4, "Shahnameh.txt") 
m.random_text(16)

'whose rule everything exists from the least straw to elephants and lions a fool , '

In [102]:
# No random seeds, so your results may vary
m = create_ngram_model(4, "Zahhak.txt") 
m.random_text(16)

'بباید بدین بود همداستان <END> کسی کاو هوای فریدون کند <END>  <END> ببخشیدشان جامه و'

In [103]:
# No random seeds, so your results may vary
m = create_ngram_model(4, "Akhavan.txt") 
m.random_text(16)

'ولیکن عزت آزادگی را <END> دو دشمن در کمین ماست ، دایم <END> نگهبانیم ، آزادیم'

## part g

In [104]:
m = NgramModel(1)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
m.perplexity("کاوه آهنگر")

3.815714141844439

In [105]:
m = NgramModel(2)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
m.perplexity("کاوه آهنگر")


1.4422495703074083